In [1]:
import pandas
from tqdm.auto import tqdm

c:\Users\MSI\.virtualenvs\Movies_Assistant-0P3PgTCo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pandas.read_csv('../data/movies.csv', keep_default_na=False)

In [4]:
documents = df.to_dict(orient='records')

In [5]:
documents[500]

{'id': 967493,
 'title': 'Inside The Walking Dead Season 11 (Part 1)',
 'genres': '',
 'original_language': 'en',
 'overview': 'Join host Clarke Wolfe for the ultimate backstage pass to the final season of The Walking Dead (Part 1) featuring never-before-seen footage access to your favorite cast and crew and sneak peeks of what’s coming next.',
 'popularity': 114.827,
 'production_companies': '',
 'release_date': '1970-01-01',
 'budget': 0,
 'revenue': 0,
 'runtime': 0,
 'status': 'Released',
 'tagline': 'Join host Clarke Wolfe for the ultimate backstage pass to the final season of The Walking Dead (Part 1), featuring never-before-seen footage, access to your favorite cast and crew, and sneak peeks of what’s coming next.',
 'vote_average': 10.0,
 'vote_count': 4,
 'credits': 'Clarke Wolfe',
 'keywords': ''}

In [6]:
len(documents)

722359

In [17]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [19]:
if es_client.indices.exists(index='movies'):
    es_client.indices.delete(index='movies')

In [20]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "id": {"type": "integer", "null_value": 0},
            "title": {"type": "text"},
            "genres": {"type": "text"},
            "original_language": {"type": "keyword"},
            "overview": {"type": "text"},
            "popularity": {"type": "float"},
            "production_companies": {"type": "text"},
            "release_date": {"type": "date", "format": "yyyy-MM-dd", "null_value": "1970-01-01"},
            "budget": {"type": "float"},
            "revenue": {"type": "float"},
            "runtime": {"type": "float"},
            "status": {"type": "keyword"},
            "tagline": {"type": "text"},
            "vote_average": {"type": "float"},
            "vote_count": {"type": "float"},
            "credits": {"type": "text"},
            "keywords": {"type": "text"},
        }
    }
}



index_name = "movies"

es_client.indices.create(index=index_name, body=index_settings) 

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'})

In [21]:
#  took so long to index, about 20it with 720k rows, ~ 600 minutes
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 277/722359 [00:16<12:04:43, 16.61it/s]


KeyboardInterrupt: 